In [ ]:
import tweepy
from tweepy import Stream
from tweepy import StreamListener
from tweepy import OAuthHandler
import json
import numpy as np
import jsonpickle
import time 

try:
    with open(".secrets", "r") as f:
        api_keys = f.readlines()
    except Error as e:
        print("no secrets file or error")
        exit()
    
consumer_key = api_keys[0]
consumer_secret = api_keys[1]

access_token = api_keys[2]
access_secret = api_keys[3]

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
# bounding_box = [19.1686845, 59.675274, 19.1686845, 70.0922241, 31.586712, 70.0922241, 31.586712, 59.675274, 19.1686845, 59.675274 ]

In [ ]:
tweetcount = 0

with open("tweets2.json", "a") as f:
    cursor = tweepy.Cursor(api.search, q="*", lang="fi", since_id=id, tweet_mode="extended").items(1000000)
    
    while True:
        try:
            tweet = cursor.next()
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + "\n")
        except tweepy.TweepError as e:
            print("err: " + str(e))
            time.sleep(60 * 15)
            continue
